In [1]:
!pip install lightning torchvision einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import lightning as L
import pandas as pd
import torchvision
import numpy as np
import torch.nn.functional as F  
from torch.autograd import Variable
import math
from functools import partial
import pathlib
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_video
import lightning as L
from lightning.pytorch.loggers import CSVLogger
import torchmetrics
from lightning.pytorch.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from torchvision.transforms import CenterCrop, v2
from datetime import datetime
import csv
import glob
import os

In [3]:
id2Label = ['[i]', 'BackSpace', ',', '[s]', '.', 
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
            'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
            'y', 'z']
label2Id  = {label: i for i, label in enumerate(id2Label)}

NUM_WORKERS = 4
f_after = 2 # number of frames after
f_before = 2 # number of frames before
gap = 2 # gap between idle video segment and non-idle video segment
total_window = f_after + f_before + 1

# Resnet

In [4]:
#### RESNET 3D #### 
def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(out.size(0), planes - out.size(1), out.size(2), out.size(3), out.size(4) ).zero_()
    
    if isinstance(out.data, torch.cuda.FloatStorage): zero_pads = zero_pads.cuda()
    out = Variable(torch.cat([out.data, zero_pads], dim=1))
    return out

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)

        self.conv2 = nn.Conv3d(
            planes, planes, 
            kernel_size=3, stride=stride, padding=1, 
            bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, sample_size, sample_duration, shortcut_type='B', num_classes=400):
        """
        block: basic block or bottle neck
        layers: define Resnet architecture 34, 101, 152 etc
        sample size: image size
        shortcut_type: 'A' or 'B'
        num_classes: ...
        """
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=7, stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], shortcut_type, stride=2)
        
        
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)
        
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(self.inplanes,planes * block.expansion,kernel_size=1,stride=stride,bias=False), 
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet10(**kwargs): return ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
def resnet18(**kwargs): return ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
def resnet34(**kwargs): return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
def resnet50(**kwargs): return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
def resnet101(**kwargs): return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
def resnet152(**kwargs): return ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
def resnet200(**kwargs): return ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)

# Transformer

In [5]:
"""
Adapted from 2020 Ross Wightman
https://github.com/rwightman/pytorch-image-models
"""
import torch
from timm.models.layers import trunc_normal_
from timm.models.vision_transformer import _load_weights
import torch.nn as nn
from einops import rearrange
from pathlib import Path
import torch.nn.functional as F
from timm.models.layers import DropPath
import torch

def init_weights(m):
    if isinstance(m, nn.Linear):
        trunc_normal_(m.weight, std=0.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
        nn.init.constant_(m.bias, 0)
        nn.init.constant_(m.weight, 1.0)
        
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout, out_dim=None):
        super().__init__()
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.act = nn.GELU()
        if out_dim is None:
            out_dim = dim
        self.fc2 = nn.Linear(hidden_dim, out_dim)
        self.drop = nn.Dropout(dropout)

    @property
    def unwrapped(self):
        return self

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Attention(nn.Module):
    def __init__(self, dim, heads, dropout):
        super().__init__()
        self.heads = heads
        head_dim = dim // heads
        self.scale = head_dim ** -0.5
        self.attn = None

        self.qkv = nn.Linear(dim, dim * 3)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout)

    @property
    def unwrapped(self):
        return self

    def forward(self, x, mask=None):
        B, N, C = x.shape
        qkv = (
            self.qkv(x)
            .reshape(B, N, 3, self.heads, C // self.heads)
            .permute(2, 0, 3, 1, 4)
        )
        
        q, k, v = (
            qkv[0],
            qkv[1],
            qkv[2],
        )

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
       

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
    
        return x, attn

class Block(nn.Module):
    def __init__(self, dim, heads, mlp_dim, dropout, drop_path):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.attn = Attention(dim, heads, dropout)
        self.mlp = FeedForward(dim, mlp_dim, dropout)
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

    def forward(self, x, mask=None, return_attention=False):
        x = self.norm1(x)

        y, attn = self.attn(x, mask)
        if return_attention:
            return attn
        x = x + self.drop_path(y)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        
        return x

class VisionTransformer(nn.Module):
    def __init__(
        self,
        frames,
        n_layers,
        d_model,
        d_ff,
        n_heads,
        n_cls,
        dropout=0.1,
        drop_path_rate=0.0,
        distilled=False,
    ):
        super().__init__()
#         self.patch_embed = PatchEmbedding(
#             image_size,
#             patch_size,
#             d_model,
#             channels,
#         )
        # self.patch_size = patch_size
        self.n_layers = n_layers
        self.d_model = d_model
        self.d_ff = d_ff
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout)
        self.n_cls = n_cls

        # cls and pos tokens
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        self.distilled = distilled
        if self.distilled:
            self.dist_token = nn.Parameter(torch.zeros(1, 1, d_model))
            self.pos_embed = nn.Parameter(
                torch.randn(1, frames + 2, d_model)
            )
            self.head_dist = nn.Linear(d_model, n_cls)
        else:
            self.pos_embed = nn.Parameter(
                torch.randn(1, frames + 1, d_model)
            )

        # transformer blocks
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, n_layers)]
        self.blocks = nn.ModuleList(
            [Block(d_model, n_heads, d_ff, dropout, dpr[i]) for i in range(n_layers)]
        )

        # output head
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, n_cls)

        trunc_normal_(self.pos_embed, std=0.02)
        trunc_normal_(self.cls_token, std=0.02)
        if self.distilled:
            trunc_normal_(self.dist_token, std=0.02)
        self.pre_logits = nn.Identity()

        self.apply(init_weights)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {"pos_embed", "cls_token", "dist_token"}

    @torch.jit.ignore()
    def load_pretrained(self, checkpoint_path, prefix=""):
        _load_weights(self, checkpoint_path, prefix)

    def forward(self, x, return_features=False):
        B, N, C, H, W = x.shape
        x = x.flatten(2)
#         print(x.shape)
   
        cls_tokens = self.cls_token.expand(B, -1, -1)
#         print(cls_tokens.shape)
        if self.distilled:
            dist_tokens = self.dist_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, dist_tokens, x), dim=1)
        else:
            x = torch.cat((cls_tokens, x), dim=1)
#         print(x.shape)
        pos_embed = self.pos_embed
#         print(pos_embed.shape)
        x = x + pos_embed
        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        if self.distilled:
            x, x_dist = x[:, 0], x[:, 1]
            x = self.head(x)
            x_dist = self.head_dist(x_dist)
            x = (x + x_dist) / 2
        else:
            x = x[:, 0]
            x = self.head(x)
        return x

vit = VisionTransformer(frames=total_window, n_layers=20, d_model=2*21*3, d_ff=256, n_heads=6, n_cls=len(id2Label))

# Lightning dataset

In [6]:
class KeyDataset(torch.utils.data.Dataset):
    def __init__(self, video_name, labels_dir, videos_dir):
        segments = []
        # Infer idle frames.
        self.labels_dir = labels_dir
        self.videos_dir = videos_dir
        df = pd.read_csv(f'{self.labels_dir}/{video_name}.csv')
        for index, row in df.iterrows():
            key_frame = int(row['Frame'])  # Frame number where key was pressed
            key_value = row['Key']  # Key pressed
            if key_value not in id2Label:
                key_value = '[s]'
            
            is_idle_before = False
            if index == 0:
                pos_start = max(key_frame - f_before, 0)
                pos_end = key_frame + f_after
                neg_start = 0
                neg_end = pos_start - gap
                is_idle_before = True
            else:
                prev_key_frame = df.iloc[index - 1]['Frame']
                pos_start = max(key_frame - f_before, 0)
                pos_end = key_frame + f_after
                prev_pos_end = prev_key_frame + f_after
                if (pos_start - prev_pos_end) - 1 >= (f_after + f_before + 1 + gap * 2):
                    neg_start =  prev_pos_end + gap
                    neg_end = pos_start - gap
                    is_idle_before = True
            
            
            # Negative class video segments before
            if is_idle_before:
                j = neg_start
                while (j + total_window - 1) <= neg_end:
                    segments.append(([j, j + total_window - 1], "[i]"))
                    j += total_window
            
            # Current video with keystroke
            segments.append(([pos_start, pos_end], key_value))
        
        self.landmarks = torch.load(f'{videos_dir}/{video_name}.pt')
        # print(self.landmarks.shape)
        self.segments = segments
    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        (start, end), label = self.segments[idx]
        return self.landmarks[start:end+1], label2Id[label]
    
    def get_class_counts(self):
        labels = [segment[1] for segment in self.segments]
        unique_elements, counts = np.unique(labels, return_counts=True)
        occurrences = dict(zip(unique_elements, counts))
        weights = np.zeros(len(id2Label))
        for label, count in occurrences.items():
            weights[label2Id[label]] = count
        return weights

class KeyDataModule(L.LightningDataModule):
    def __init__(self, batch_size, labels_dir, videos_dir, train_vids, val_vids, test_vids):
        super().__init__()
        self.batch_size = batch_size
        self.train_datasets = [KeyDataset(video_name, labels_dir, videos_dir) for video_name in train_vids]
        self.val_datasets = [KeyDataset(video_name, labels_dir, videos_dir) for video_name in val_vids]
        self.test_datasets = [KeyDataset(video_name, labels_dir, videos_dir) for video_name in test_vids]
        
        self.train_dataset = torch.utils.data.ConcatDataset(self.train_datasets)
        self.test_dataset = torch.utils.data.ConcatDataset(self.test_datasets)
        self.val_dataset = torch.utils.data.ConcatDataset(self.val_datasets)
        
        print(f"Train: {len(self.train_dataset)}; Val: {len(self.val_dataset)}; Test: {len(self.test_dataset)}")
        
        train_counts = np.array(
            [d.get_class_counts() for d in self.train_datasets]).sum(axis=0)
        print(f"Train counts: {train_counts}")
        train_total_samples = np.array([len(d) for d in self.train_datasets]).sum(axis=0)
        self.train_weights = train_counts / (train_total_samples * len(id2Label))
                                        
    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size, 
                          num_workers=NUM_WORKERS)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size, 
                          num_workers=NUM_WORKERS,
                          shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size, 
                          num_workers=NUM_WORKERS,
                          shuffle=False)

class KeyClf(L.LightningModule):
    def __init__(self, model, num_classes, learning_rate, weights):
        super().__init__()
        self.model = model
        
        self.loss_fn = torch.nn.CrossEntropyLoss(torch.tensor(weights).float())
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.lr = learning_rate
#         self.transforms = v2.Compose([
#             v2.CenterCrop(img_size),
#             v2.ToDtype(torch.float32, scale=True),
#         ])
        
        self.test_preds = []
        self.test_targets = []
        self.save_hyperparameters(ignore=['model'])

    def _common_step(self, batch):
        videos, targets = batch
        preds = self.model(videos)
        pred_ids = torch.argmax(self.model(videos), dim=1)
        loss = self.loss_fn(preds, targets.long())
        acc = self.accuracy(preds, targets)
        return loss, acc, pred_ids
    
    def test_step(self, batch):
        videos, targets = batch
        loss, acc, pred_ids = self._common_step(batch)
        pred_labels = [id2Label[_id] for _id in pred_ids]
        self.test_preds += pred_labels
        self.test_targets += [id2Label[_id] for _id in targets]
        self.log_dict({'test_loss': loss, 'test_acc': acc})

    
    def on_test_end(self):
        print(classification_report(self.test_targets, self.test_preds))
        
    def training_step(self, batch):
        loss, acc, _ = self._common_step(batch)
        self.log_dict({'train_loss': loss, 'train_acc': acc})
        return loss

    def validation_step(self, batch):
        loss, acc, _ = self._common_step(batch)
        self.log_dict({'val_loss': loss, 'val_acc': acc})
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [7]:
test_video_1 = KeyDataset(
    'video_1',
    labels_dir="/kaggle/input/landmarks/labels/labels",
    videos_dir="/kaggle/input/landmarks"
)
test_video_1[0][0].shape

torch.Size([5, 2, 21, 3])

## Train

In [8]:
dm = KeyDataModule(batch_size=8, 
                   labels_dir="/kaggle/input/landmarks/labels/labels",
                    videos_dir="/kaggle/input/landmarks",
                   train_vids=[
                       'video_1', 'video_2', 'video_3', 'video_4', 'video_5', 
                        'video_6', 'video_7', 'video_8', 'video_9', 'video_10',
                       'video_11', 'video_12', 'video_13', 'video_14', 'video_15', 
                       'video_16', 'video_17', 'video_18', 'video_19',
                       'video_21', 'video_22', 'video_23', 'video_24', 'video_25', 
                       'video_26', 'video_27', 'video_28', 'video_29', 'video_30'], 
                   val_vids=['video_31', 'video_32', 'video_33'], 
                   test_vids=['video_34','video_35', 'video_36'])
# resnet = resnet152(sample_size=img_size, sample_duration=total_window, shortcut_type='B', num_classes=len(id2Label))

model = KeyClf(vit, num_classes=len(id2Label), 
               learning_rate=0.001,
               weights=dm.train_weights)
logger = CSVLogger("logs", name=f"vit", flush_logs_every_n_steps=1)
trainer = L.Trainer(
    # deterministic=True,
    devices=[0],
    accelerator="gpu",
    fast_dev_run=True,
    callbacks=[EarlyStopping(monitor="val_loss", patience=10)],
    logger=logger,
    max_epochs=10,
    # enable_progress_bar=False
)
trainer.fit(model, dm)
trainer.test(model, dm)

Train: 77589; Val: 7168; Test: 4377
Train counts: [29360.  4182.   532.  7038.   376.  2938.   590.  1222.  1179.  4348.
   682.   734.  1228.  2825.   407.   363.  1573.   850.  2352.  2539.
   917.   346.  2248.  1843.  2776.  1417.   405.   587.   497.   779.
   456.]


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | VisionTransformer  | 2.6 M  | train
1 | loss_fn  | CrossEntropyLoss   | 0      | train
2 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.373    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_steps=1` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

         [i]       1.00      1.00      1.00         8

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            1.0            │
│         test_loss         │    1.3589211702346802     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.3589211702346802, 'test_acc': 1.0}]

## Resume training state

In [9]:
# dm = KeyDataModule(batch_size=8, 
#                    labels_dir='/kaggle/input/keystroke/labels',
#                    videos_dir="/kaggle/input/keystroke/raw_frames_320",
#                    train_vids=[
#                        'video_1', 'video_2', 'video_3', 'video_4', 'video_5', 
#                         'video_6', 'video_7', 'video_8', 'video_9', 'video_10',
#                        'video_11', 'video_12', 'video_13', 'video_14', 'video_15', 
#                        'video_16', 'video_17', 'video_18', 'video_19',
#                        'video_21', 'video_22', 'video_23', 'video_24', 'video_25', 
#                        'video_26', 'video_27', 'video_28', 'video_29', 'video_30'], 
#                    val_vids=['video_31', 'video_32', 'video_33'], 
#                    test_vids=['video_34', 'video_35', 'video_36'])
# model = KeyClf.load_from_checkpoint("/kaggle/input/key-clf-smaller/pytorch/v1/1/smaller-epoch10-step53350.ckpt")
# trainer = L.Trainer(
#     # deterministic=True,
#     accelerator="tpu",
#     fast_dev_run=False,
#     max_time="00:11:00:00",
#     callbacks=[EarlyStopping(monitor="val_loss", patience=10)]
# )
# trainer.fit(model, dm)
# trainer.test(model, dm)

## Test

In [10]:
# trained_model = KeyClf.load_from_checkpoint("/kaggle/input/key-clf-smaller/pytorch/v1/1/smaller-epoch10-step53350.ckpt")

# def levenshtein_distance(s1, s2):
#     # Initialize a matrix with zeros
#     dp = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
    
#     # Initialize first row and column
#     for i in range(len(s1) + 1):
#         dp[i, 0] = i
#     for j in range(len(s2) + 1):
#         dp[0, j] = j
    
#     # Fill the matrix
#     for i in range(1, len(s1) + 1):
#         for j in range(1, len(s2) + 1):
#             cost = 0 if s1[i - 1] == s2[j - 1] else 1
#             dp[i, j] = min(dp[i - 1, j] + 1,      # deletion
#                            dp[i, j - 1] + 1,      # insertion
#                            dp[i - 1, j - 1] + cost)  # substitution
    
#     # Return the edit distance
#     return dp[len(s1), len(s2)]

# def similarity_percentage(s1, s2):
#     # Calculate the Levenshtein distance
#     distance = levenshtein_distance(s1, s2)
#     # Calculate the maximum possible distance
#     max_distance = max(len(s1), len(s2))
#     # Calculate similarity as a percentage
#     similarity = 1 - distance / max_distance  
#     # Convert to percentage
#     similarity_percentage = similarity * 100
#     return similarity_percentage

# def process_prediction(preds, prob_min = 0.5):
#     pred_text = []
#     all_keys = preds.iloc[:,1].to_list()
#     all_probs = preds.iloc[:, 2].to_list()
#     i = 0
#     while i < len(preds):
#         # Get all the similar keys next to it
#         curr_key = all_keys[i]
#         # print(''.join(pred_text))
#         # print('curr_key: ', curr_key)
#         j = i + 1
#         while j < len(preds):
#             if all_keys[j] != curr_key: 
#                 j = j - 1
#                 break
#             j += 1
        
#         occurences = j - i + 1
#         # print('occurences: ', occurences)
        
#         # Idle keys => skip
#         if curr_key == '[i]':
#             i = j + 1
#             continue

#         # Backspace => count how many consecutive bs keys. For 4, remove one prev key 
#         elif curr_key == 'BackSpace':
#             deletions = occurences // 4
            
#             for _ in range(deletions):
#                 if len(pred_text):
#                     pred_text.pop()
            

#         # Other character, less than 4 occurces append once. 
#         # Else, for the every next 2 occurrence append once (because people press 2 same keys faster)
#         else:
#             if curr_key == '[s]': curr_key = " "
#             elif curr_key == '[,]': curr_key = ','
#             elif curr_key == '[.]': curr_key = '.'
            
#             if occurences == 1: 
#                 if all_probs[i] >= prob_min:
#                     pred_text.append(curr_key)
                
            
#             elif occurences <= 4:
#                 if max(all_probs[i:j+1]) >= prob_min:
#                     pred_text.append(curr_key)
                
#             else:
#                 pred_text.append(curr_key)
#                 for _ in range((occurences - 4) // 4):
#                     pred_text.append(curr_key)
        
#         i = j + 1
    

#     return ''.join(pred_text)

# def test_video(video_dir, ground_truth_dir, video_name):
#     images = glob.glob(f'{video_dir}/{video}/*.jpg')
#     frames = []
#     for i in range(len(images)):
#         image = torchvision.io.read_image(f"{video_dir}/{video_name}/frame_{i}.jpg")
#         frames.append(image)
#     frames = torch.stack(frames)

#     keys = []
#     frame_no = []
#     probs = []
#     trained_model.freeze()
#     for i in range(0, len(frames)):
#         video = trained_model.transforms(frames[i: i + 5])
#         video = video.permute(1, 0, 2, 3)
#         out = F.softmax(trained_model.model(video.unsqueeze(0)), 1)[0]
#         _id = torch.argmax(out).item()
#         label = id2Label[_id]
        
#         print(f"Frame {i};{label};{out[_id]}")
#         frame_no.append(i)
#         keys.append(label)
#         probs.append(out[_id])
    
#     df = pd.DataFrame({
#         'Frame': frame_no,
#         'Key': keys,
#         'Prob': probs,
#     })
#     os.makedirs('./test-result', exist_ok=True)
#     df.to_csv(f'./test-result/{video_name}.csv', sep=';', index=False)

#     pred_text = process_prediction(df)
#     print('pred_text: ', pred_text)
    
#     if ground_truth_dir:
#         ground_truth = open(f'{ground_truth_dir}/{video_name}.txt', 'r').read()
#         similarity = similarity_percentage(pred_text, ground_truth)
#         print('similarity: ', similarity)
    
#     open(f"./test-result/{video_name}_processed.txt", 'w').write(pred_text)

# test_video(video_dir = '/kaggle/input/keystroke/raw_frames_320', 
#            video='video_36',
#            ground_truth_dir = '/kaggle/input/keystroke/ground_truths')

In [11]:
# pred_text = 'dr team, i op ouou al ha  fnnasatic time t our recent empohye pprcition ent  t s ta tryuuuly truy a plarsr to s eerne scome to ggehr nd nouy fte estivities . our entunnusism nd ositi enrgy md th ent a resounding succes   we nt ed to ta e  moment to eextend our  herrt  et grrduttidtde to ec of o r hrd ror and ddicttiion  cbent s e thes r sa sml tlon of pprcition or  thde incrdil eort uou put intol ouyourrr ros eer  d d s e mo forrd  s crrt forrdd ths sprit of cmrderies  nd temor  o r contriutions r inuatl to our commpny succs  and  loo orwrd to chingieing  eem greate moile satoonnes gtotogter once gain  , thnn uo fo bining an intergra  prt of our tem here s to continu d scscuc c es nd mn  more mmorae moments esd b st reggards t'
# ground_truth = open('/kaggle/input/keystroke/ground_truths/video_35.txt', 'r').read()
# similarity = similarity_percentage(pred_text, ground_truth)
# similarity

In [12]:
# corrected = "dr team, i hope you all had a fantastic time at our recent employee appreciation event. it was truly a pleasure to see everyone come together and enjoy the festivities. your enthusiasm and positive energy made the event a resounding success. we wanted to take a moment to extend our heartfelt gratitude to each of you for your hard work and dedication. events like these are a small token of appreciation for the incredible effort you put into your roles every day. we look forward to carrying forward this spirit of camaraderie and teamwork. your contributions are invaluable to our company's success, and we look forward to achieving even greater milestones together once again. thank you for being an integral part of our team. here's to continued success and many more memorable moments. best regards."
# similarity = similarity_percentage(corrected, ground_truth)
# similarity